<a href="https://colab.research.google.com/github/ishitajoshi05/BOOK-GENRE-PREDICTION/blob/main/BOOKGENRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import ast
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# ---------------------------
# Helper functions
# ---------------------------

def clean_text(text):
    """
    Basic text cleaning: lowercasing and removing non-alphabetic characters.
    """
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def convert_genre_ratios(ratio_str):
    """
    Convert string representation of dictionary to an actual dictionary.
    """
    try:
        return ast.literal_eval(ratio_str)
    except Exception as e:
        return {}

def create_target_dataframe(df, target_column='Genre_Ratios'):
    """
    Convert the Genre_Ratios column into a dataframe with one column per genre.
    Missing genre values are filled with 0.
    """
    # Convert string to dictionary for each row
    df['Genre_Ratios_dict'] = df[target_column].apply(convert_genre_ratios)

    # Get the union of all genres present in the dataset
    all_genres = set()
    for d in df['Genre_Ratios_dict']:
        all_genres.update(d.keys())
    all_genres = sorted(list(all_genres))

    # Create new columns for each genre
    for genre in all_genres:
        df[genre] = df['Genre_Ratios_dict'].apply(lambda d: d.get(genre, 0))

    return df, all_genres

def build_and_train_model(df, genre_columns, text_column='Synopsis'):
    """
    Build TF-IDF features from text and train a multi-output regression model.
    """
    # Clean text
    df['Cleaned_Text'] = df[text_column].apply(clean_text)

    # Extract TF-IDF features
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(df['Cleaned_Text'])

    # Prepare target matrix
    y = df[genre_columns].values.astype(float)

    # Split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train the model using a MultiOutputRegressor
    base_model = RandomForestRegressor(n_estimators=100, random_state=42)
    model = MultiOutputRegressor(base_model)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Test Mean Squared Error:", mse)

    return model, vectorizer

def predict_genre_ratios(synopsis, model, vectorizer, genre_columns):
    """
    Given a new book synopsis, preprocess it, extract TF-IDF features,
    and predict the genre ratios.
    """
    cleaned = clean_text(synopsis)
    X_new = vectorizer.transform([cleaned])
    y_pred = model.predict(X_new)[0]

    # Optionally, convert to percentages (summing to 100)
    total = y_pred.sum()
    if total > 0:
        y_pred_pct = (y_pred / total) * 100
    else:
        y_pred_pct = y_pred

    # Create a dictionary of predictions
    prediction = {genre: round(float(ratio), 2) for genre, ratio in zip(genre_columns, y_pred_pct)}
    return prediction

# ---------------------------
# Main pipeline
# ---------------------------

# Load the synthetic dataset
df = pd.read_csv("large_book_genre_dataset.csv")

# Create target columns from Genre_Ratios
df, genre_columns = create_target_dataframe(df, target_column='Genre_Ratios')
print("Target genres:", genre_columns)

# Build and train the model
model, vectorizer = build_and_train_model(df, genre_columns, text_column='Synopsis')

# Example: Predict genre ratios for a new book synopsis
new_synopsis = "In a futuristic world, humans travel across galaxies, facing political intrigues and unexpected romances among alien civilizations."
predicted_ratios = predict_genre_ratios(new_synopsis, model, vectorizer, genre_columns)
print("Predicted Genre Ratios (in percentages):", predicted_ratios)
